# Ray Test Script

This notebook contains a series of tests to verify that Ray is working correctly on your system.

In [ ]:
# Install ray if not installed

In [ ]:
import os
import time
import numpy as np
import ray

# Initialize Ray
ray.init()
print(f"Ray initialized with the following resources:\n{ray.available_

In [ ]:
# Define a remote function
@ray.remote
def simple_function(x):
    return x * x

# Execute the function remotely
futures = [simple_function.remote(i) for i in range(5)]
results = ray.get(futures)
print("Remote function results:", results)

In [ ]:
# Define a function that takes some time to complete
@ray.remote
def slow_function(i):
    time.sleep(1)  # Simulate work taking 1 second
    return i * 10

# Execute in parallel
start_time = time.time()
futures = [slow_function.remote(i) for i in range(10)]
results = ray.get(futures)
end_time = time.time()

print(f"Parallel execution results: {results}")
print(f"Parallel execution time: {end_time - start_time:.2f} seconds")

# Execute sequentially for comparison
start_time = time.time()
sequential_results = []
for i in range(10):
    sequential_results.append(i * 10)
    time.sleep(1)  # Simulate work taking 1 second
end_time = time.time()

print(f"Sequential execution results: {sequential_results}")
print(f"Sequential execution time: {end_time - start_time:.2f} seconds

In [ ]:
# Define an actor
@ray.remote
class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

    def get_value(self):
        return self.value

# Create an actor
counter = Counter.remote()

# Call methods on the actor
futures = [counter.increment.remote() for _ in range(5)]
results = ray.get(futures)
print("Actor method results:", results)

# Get the final counter value
final_value = ray.get(counter.get_value.remote())

In [ ]:
# Create a large array
@ray.remote
def process_chunk(chunk):
    # Simulate some processing on a chunk of data
    return np.sum(chunk)

# Create a large array (100 million elements)
data_size = 10_000_000
large_array = np.ones(data_size, dtype=np.float32)

# Split into chunks and process in parallel
num_chunks = 10
chunk_size = data_size // num_chunks
chunks = [large_array[i:i+chunk_size] for i in range(0, data_size, chunk_size)]

# Process chunks in parallel
start_time = time.time()
futures = [process_chunk.remote(chunk) for chunk in chunks]
chunk_sums = ray.get(futures)
total_sum = sum(chunk_sums)
end_time = time.time()

print(f"Parallel sum: {total_sum}")
print(f"Parallel processing time: {end_time - start_time:.4f} seconds")

# Process sequentially for comparison
start_time = time.time()
sequential_sum = sum(np.sum(chunk) for chunk in chunks)
end_time = time.time()

print(f"Sequential sum: {sequential_sum}")
print(f"Sequential processing time: {end_time - start_time:.4f} seconds")

In [ ]:
# Clean up Ray resources
ray.shutdown()
print("Ray resources released.")